In [1]:
# Generated code -- http://www.redblobgames.com/grids/hexagons/
import collections
import math, random
import itertools
import os, copy
from copy import deepcopy
import time

def initializeBoard():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.randint(0,1)
            z = -x-y
            Hex[(x, y, z)] = ''#side[rnd]
    return Hex

def allWhiteBoardZobrist():
    whitezobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            whitezobristboard[(x, y, z)] = rnd
    return whitezobristboard

def allBlackBoardZobrist():
    blackzobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            blackzobristboard[(x, y, z)] = rnd
    return blackzobristboard

def initializeBoardZobrist():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            Hex[(x, y, z)] = rnd
    return Hex

def boardToZobrist(board, zboard, wzobristboard, bzobristboard):
    random.seed(1356)
    zobristkey = random.getrandbits(64)
    wzobrists = wzobristboard.values()
    bzobrists = bzobristboard.values()
    
    for cel in board:
        if board[cel] == 'b':
            zobristkey ^= (bzobristboard[cel] ^ zboard[cel])
        elif board[cel] == 'w':
            zobristkey ^= (wzobristboard[cel] ^ zboard[cel])
        
    return zobristkey

In [2]:
def createTT(fromZobrist, toZobrist, depth, score, flag, eval):
    tt = {(0,9,8): [toZobrist, depth, score, flag, eval]}
    
    return fromZobrist % 1000


rnd = random.getrandbits(64)

createTT(rnd, rnd, 0, 999, -1, 0)

508

In [3]:
# Form groups and calculate nr of members
def EvalBoard(board):
    score = find_scores(board, 'b') #score of player1
    score2 = find_scores(board, 'w') #score of player2

    group = findGroups(score)
    group2 = findGroups(score2)

    res1, res2 = result(group, group2)
    return res1, res2, res1-res2


def find_scores(board, side):   
    tot = {}
    
    listOfkeys = [key for (key, value) in board.items() if value == side]
    
    for lok  in listOfkeys:
        tot[lok] = side
        
    return tot

def listOfEmptySpots(board):
    return [k for k, v in board.items() if v == '']

def neighbours(tiles, scoregroup, visited):
    group = []
    for tile in tiles:
        for direction in hex_directions:
            new_tile2 = tuple(map(sum, zip(tile, direction)))
            if new_tile2 in scoregroup and new_tile2 not in visited:
                group += [new_tile2]
                visited += [new_tile2]
    return group, visited

def findGroups(score):
    newlist = []
    visited = []
    for sc in score:
        temp = [sc]
        if sc not in visited:
            visited += [sc]
            groupt, visited = neighbours([sc], score, visited)
            temp += groupt
            visited += groupt
            find = True
            while find:
                groupt, visited = neighbours(groupt, score, visited)
                if len(groupt) != 0:
                    temp += groupt
                else:
                    find = False
                    
            newlist += [len(temp)]
    return newlist


# multiply all group counts of the same color for final game score
from functools import reduce

def multiplyall(count_list):
    if count_list != []:
        return reduce(lambda x, y: x*y, count_list)
    return 0

def result(count_list, count_list2):
    game_total = multiplyall(count_list)
    game_total2 = multiplyall(count_list2)
    return game_total, game_total2

# Put pieces on board
def makeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
    return board

# Put pieces on board
def unmakeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
    return board

def isEmpty(board, position):
    if position in board:
        return len(board[position]) == 0
    else:
        return False

def fullBoard(board):
    number_ofcells = len(board)
    number_ofstones = 0
    for cell in board:
        if board[cell] != '':
            number_ofstones += 1
    return number_ofstones >= number_ofcells -1

#Check if a move is valid
def validMove(board, x, y):
    
    #Check if there is a piece on the location
    if not isEmpty(board, (x, y, -x-y)):
        return False
    
    #Check if the piece is on the board
    if abs(x) > N or abs(y) > N:
        return False
    
    return True

In [4]:
def moves_left(board):
    print('These are the moves left: ')
    
    moves = ''
    counter = 0
    for pos in board:
        
        if board[pos] == '':
            moves += str(pos)
            counter += 1
            if counter == 7:
                moves += '\n'
                counter = 0
            else:
                moves += ' '
    print (moves)

In [5]:
def zobristStateScore(board):

    # Create a zobrist key for the new board/childnode
    zidentity = boardToZobrist(board, zboard, wzobristboard, bzobristboard)
    game_state, tempscore = 0, 0
    # If the board is full, tell the score
    if fullBoard(board):

        score = find_scores(board, 'b') #score of player1
        score2 = find_scores(board, 'w') #score of player2

        count_list = findGroups(score)
        count_list2 = findGroups(score2)

        player_one_total, player_two_total = result(count_list, count_list2)

        # Did player 1 win, lose or draw?
        game_state = (('l', 'w')[player_one_total < player_two_total ], 'd')[player_one_total == player_two_total]
        tempscore = ((-999, 999)[player_one_total < player_two_total ], -499)[player_one_total == player_two_total]
        
    return zidentity, game_state, tempscore

In [6]:
random.seed(1356) # set seed so that each play is the same

hex_directions = [(1, 0, -1), (1, -1, 0), (0, -1, 1), (-1, 0, 1), (-1, 1, 0), (0, 1, -1)]
hex_diagonals = [(2, -1, -1), (1, -2, 1), (-1, -1, 2), (-2, 1, 1), (-1, 2, -1), (1, 1, -2)]

side = [random.getrandbits(64), random.getrandbits(64)]#[b, w, empty] # side to move ['b', 'w']#

N = 1 # Number of hexes side
p = 2 # number of players

blackstones = 136
whitestones = 136

#Game ends if number of free tiles < 4
total_nr_stones = blackstones + whitestones <4

board = initializeBoard() # board with assigned black and white pieces
zboard = initializeBoardZobrist() # initial board, without any pieces

wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

INFINITY = 999


# board#, len(board)

In [7]:
board = initializeBoard()
score = find_scores(board, 'b') #score of player1
score2 = find_scores(board, 'w') #score of player2

count_list = findGroups(score)
count_list2 = findGroups(score2)
result(count_list, count_list2)

(0, 0)

In [8]:
def AlphaBeta(board, depth, alpha, beta, maximizingPlayer):
    if depth == 0 or fullBoard(board):
        if maximizingPlayer:
            return EvalBoard(board)[0]
        else:
            return EvalBoard(board)[1]
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board2 = deepcopy(board) # make sure that you copy the board from the parent!!
    
    if maximizingPlayer:
        v = -INFINITY

        for cell in all_cells:

            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board2, x, y) and validMove(board2, x2, y2):

                makeMove(board2, [x, y], 'b')
                makeMove(board2, [x2, y2], 'w')
                boardTemp = board2
                v = max(v, AlphaBeta(boardTemp, depth - 1, alpha, beta, False))
                alpha = max(alpha, v)
                if beta <= alpha:
                    break # beta cut-off
        return v
    else: # minimizingPlayer
        v = INFINITY
        for cell in all_cells:

            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board2, x, y) and validMove(board2, x2, y2):

                makeMove(board2, [x, y], 'b')
                makeMove(board2, [x2, y2], 'w')
                boardTemp = board2
                v = min(v, AlphaBeta(boardTemp, depth - 1, alpha, beta, True))
                beta = min(beta, v)
                if beta <= alpha:
                    break # alpha cut-off
        return v

In [9]:
def AlphaBetaTT(board, depth, alpha, beta, maximizingPlayer):
    if depth == 0 or fullBoard(board):
        if maximizingPlayer:
            return EvalBoard(board)[0]
        else:
            return EvalBoard(board)[1]
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board2 = deepcopy(board) # make sure that you copy the board from the parent!!
    
    if maximizingPlayer:
        v = -INFINITY

        for cell in all_cells:

            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board2, x, y) and validMove(board2, x2, y2):

                makeMove(board2, [x, y], 'b')
                makeMove(board2, [x2, y2], 'w')
                boardTemp = board2
                values = zobristStateScore(boardTemp)
                v = max(v, AlphaBeta(boardTemp, depth - 1, alpha, beta, False))
                alpha = max(alpha, v)
                if beta <= alpha:
                    break # beta cut-off
        return v
    else: # minimizingPlayer
        v = INFINITY
        for cell in all_cells:

            x, y, _ = cell[0]
            x2, y2, _ = cell[1]

            #Make two valid moves
            if validMove(board2, x, y) and validMove(board2, x2, y2):

                makeMove(board2, [x, y], 'b')
                makeMove(board2, [x2, y2], 'w')
                boardTemp = board2
                values = zobristStateScore(boardTemp)
                v = min(v, AlphaBeta(boardTemp, depth - 1, alpha, beta, True))
                beta = min(beta, v)
                if beta <= alpha:
                    break # alpha cut-off
        return v

In [10]:
def NegamaxAB(board, depth, alpha, beta, side):
    if depth == 0 or fullBoard(board):
        return side * EvalBoard(board)[2]
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board2 = deepcopy(board) # make sure that you copy the board from the parent!!
    
    v = -INFINITY

    for cell in all_cells:

        x, y, _ = cell[0]
        x2, y2, _ = cell[1]

        #Make two valid moves
        if validMove(board2, x, y) and validMove(board2, x2, y2):

            makeMove(board2, [x, y], 'b')
            makeMove(board2, [x2, y2], 'w')
            boardTemp = board2
            
            v = max(v, -NegamaxAB(boardTemp, depth - 1, -alpha, -beta, -side))
            
            alpha = max(alpha, v)
            if beta <= alpha:
                break # beta cut-off
    return v

In [11]:
def Negascout(board, depth, alpha, beta, side):
    if depth == 0 or fullBoard(board):
        return side * EvalBoard(board)[2]
    
    firstChild = True
    bestValue = -INFINITY
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board2 = deepcopy(board) # make sure that you copy the board from the parent!!
    
    for cell in all_cells:

        x, y, _ = cell[0]
        x2, y2, _ = cell[1]
        
        #Make two valid moves
        if validMove(board2, x, y) and validMove(board2, x2, y2):

            makeMove(board2, [x, y], 'b')
            makeMove(board2, [x2, y2], 'w')
            
            if not firstChild:
                score = -Negascout(board2, depth - 1, -alpha - 1, -alpha, -side)
                if alpha < score and score < beta:
                    score = -Negascout(board2, depth - 1, -beta, -score, -side)
            else:
                firstChild = False
                score = -Negascout(board2, depth - 1, -beta, -alpha, -side)
            alpha = max(alpha, score)
            if alpha >= beta:
                break
    return alpha

In [12]:
def BestMove(board):
    maxPoints = 0
    mx = N+1; 
    my = N+1
    mx2 =  N+1; 
    my2 = N+1
    

    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board2 = deepcopy(board) # make sure that you copy the board from the parent!!
    
    for cell in all_cells:

        x, y, _ = cell[0]
        x2, y2, _ = cell[1]

        #Make two valid moves
        if validMove(board2, x, y) and validMove(board2, x2, y2):

            makeMove(board2, [x, y], 'b')
            makeMove(board2, [x2, y2], 'w')

            if opt == 0:
                points = NegamaxAB(board2, depth, -INFINITY, +INFINITY, 1)
            elif opt == 1:
                points = NegamaxAB(board2, depth, -INFINITY, +INFINITY, 1)
            elif opt == 2:
                points = Negascout(board2, depth, -INFINITY, +INFINITY, 1)
            elif opt == 3:
                points = AlphaBeta(board, depth, -INFINITY, +INFINITY, True)
            elif opt == 4:
                points = AlphaBetaTT(board, depth, -INFINITY, +INFINITY, True)

            if points > maxPoints:
                maxPoints = points
                mx = x; my = y
                mx2 = x2; my2 = y2
    
    
    return [mx, my], [mx2, my2], board2

In [13]:
#Creating entire the game tree for the play
from copy import deepcopy
import time
lookup = {}
zboard = initializeBoardZobrist() # initial board, without any pieces
wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

def buildGameTree2(board, depth):
    depth += 1
    zidentity0 = boardToZobrist(board, zboard, wzobristboard, bzobristboard) # current root/parent
    tempscore, game_state, zidentity = 0,0,0
    #Only make a move when a move is valid
    valid = False

    board2 = deepcopy(board) # make sure that you copy the board from the parent!!
                # If the board is full, tell the score
    if fullBoard(board2):
        # Create a zobrist key for the new board/childnode
        zidentity = boardToZobrist(board2, zboard, wzobristboard, bzobristboard)
        
        score = find_scores(board, 'b') #score of player1
        score2 = find_scores(board, 'w') #score of player2

        count_list = findGroups(score)
        count_list2 = findGroups(score2)

        player_one_total, player_two_total = result(count_list, count_list2)

        # Did player 1 win, lose or draw?
        game_state = (('l', 'w')[player_one_total < player_two_total ], 'd')[player_one_total == player_two_total]
        tempscore = ((-999, 999)[player_one_total < player_two_total ], -499)[player_one_total == player_two_total]
        return board2, zidentity, depth, tempscore
    
    while not valid: 
        x = random.randint(-N, N)
        y = random.randint(-N, N)
        x2 = random.randint(-N, N)
        y2 = random.randint(-N, N)
        
        #Make two valid moves
        if validMove(board2, x, y) and validMove(board2, x2, y2):
                
            makeMove(board2, [x, y], 'b')
            makeMove(board2, [x2, y2], 'w')
#             valid = True

            # Create a zobrist key for the new board/childnode
            zidentity = boardToZobrist(board2, zboard, wzobristboard, bzobristboard)

            # If the board is full, tell the score
            if fullBoard(board2):

                score = find_scores(board, 'b') #score of player1
                score2 = find_scores(board, 'w') #score of player2

                count_list = findGroups(score)
                count_list2 = findGroups(score2)

                player_one_total, player_two_total = result(count_list, count_list2)

                # Did player 1 win, lose or draw?
                game_state = (('l', 'w')[player_one_total < player_two_total ], 'd')[player_one_total == player_two_total]
                tempscore = ((-999, 999)[player_one_total < player_two_total ], -499)[player_one_total == player_two_total]
            if zidentity not in lookup:
                lookup[zidentity] = board2
                
            valid = True
    return board2, zidentity, depth, tempscore, (x, y), (x2, y2)

In [20]:
# def startGame(play):
points = 0
print ('Omega game initialised')
ai = ['Negamax', 'Negascout(PVS)', 'Alpha Beta regular', 'Alpha Beta TT']
menu = ['0: EvalBoard', '1: Negamax w/ Alpha-Beta Pruning', '2: Negascout (Principal Variation Search)', 
        '3: Alpha Beta regular', '4: Alpha Beta TT']

for item in menu:
    print(item)
    
opt = int(input('Select AI engine: '))
if not (opt >= 1 and opt <= 4):
    opt = int(input(str(opt) + ' is the wrong input, select your AI again. press 1, 2, 3 or 4'))
else:
    print('You have selected the', ai[opt-1], 'AI')
print ('\n1: User\n2: AI \n(Press Enter for Exit!)')
board = initializeBoard()
player = 3
depth = 5

play = True

while play:
    print()
#         PrintBoard()
    print ('The next move is for the player', player)

    if fullBoard(board):
        print ('No moves left! Game ended!')
        print ('Score User: ' + str(EvalBoard(board) [0]))
        print ('Score AI  : ' + str(EvalBoard(board) [1]))
        if EvalBoard(board) [0] > EvalBoard(board) [1]:
            print('Player b wins')
        elif EvalBoard(board) [0] == EvalBoard(board) [1]:
            print('It\'s a tie!')  
        else:
            print('AI wins')
#             os._exit(0)
        play = False

    if player == 1: # user's turn
        while True:
            xy = input('X,Y: ')
            if xy == '': os._exit(0)
            if xy == '?': 
                moves_left(board)
                xy = input('X,Y: ')
                
            (x, y) = xy.split(',')
            x = int(x)
            y = int(y)
            done = 0
            if validMove(board, x, y):
                board = makeMove(board, [x, y], 'b')
                player = 2
#                     print(board)           # Print out board
                done = 1
            else:
                print ('Invalid move! Try again!')
                continue

            xy = input('X,Y: ')
            if xy == '': os._exit(0)
            if xy == '?': 
                moves_left(board)
                xy = input('X,Y: ')
            (x, y) = xy.split(',')
            x = int(x)
            y = int(y)
            if done == 1 and validMove(board, x, y):
                board = makeMove(board, [x, y], 'w')
#                     print(board)           # Print out board
                break
            else:
                print ('Invalid move! Try again!')
    else: # AI's turn
        start = time.time()
        moveb, movew, _ = BestMove(board)
        print('Time: ', time.time() - start)
        if moveb[0] != N +1:
            makeMove(board, moveb, 'b')
            makeMove(board, movew, 'w')
            if player == 3:
                player = 4
            elif player == 4:
                player = 3
            elif player == 2:
                player = 1
    #         print(board)                 # Print out board
            print ('AI played b (X Y): ', moveb)
            print ('and w (X Y): ', movew)

        else:
            continue
    
if not play:
    print('\nGame ended!')

Omega game initialised
0: EvalBoard
1: Negamax w/ Alpha-Beta Pruning
2: Negascout (Principal Variation Search)
3: Alpha Beta regular
4: Alpha Beta TT
Select AI engine: 4
You have selected the Alpha Beta TT AI

1: User
2: AI 
(Press Enter for Exit!)

The next move is for the player 3
Time:  0.005132198333740234
AI played b (X Y):  [-1, 0]
and w (X Y):  [-1, 1]

The next move is for the player 4
Time:  0.0013878345489501953
AI played b (X Y):  [0, -1]
and w (X Y):  [0, 0]

The next move is for the player 3
Time:  0.0006089210510253906
AI played b (X Y):  [0, 1]
and w (X Y):  [1, -1]

The next move is for the player 4
No moves left! Game ended!
Score User: 2
Score AI  : 3
AI wins
Time:  0.0001220703125

Game ended!


In [ ]:
249.9475531578064
1,2


# 